In [ ]:
# Using Word2Vec embedding approach and RF, GB, and XGB to classify Decision according to Facts

In [ ]:
#mount the google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [ ]:
INPUT_EXCEL_FILE3 = "/content/drive/MyDrive/worker_comp_work/WC_Final/research_ready_Facts_train.xlsx"
INPUT_EXCEL_FILE4 = "/content/drive/MyDrive/worker_comp_work/WC_Final/research_ready_Facts_test.xlsx"

In [ ]:
#load training and testing files for Issues
df_train_facts = pd.read_excel(INPUT_EXCEL_FILE3)
df_test_facts = pd.read_excel(INPUT_EXCEL_FILE4)

In [ ]:
df_train_facts.columns

In [ ]:
df_train_facts.shape
#should be 4882

In [ ]:
df_test_facts.shape
#should be 1221

In [ ]:
#check unique values in 'Decision' column
df_train_facts['Decision'].unique()

In [ ]:
#check the distribution of Decsion variable in train and test as percentage
df_train_facts['Decision'].value_counts(normalize=True)
#should be about 64.4%-35.5%

In [ ]:
df_test_facts['Decision'].value_counts(normalize=True)

In [ ]:
train_text = df_train_facts['Preprocessed_Facts'].tolist()
test_text = df_test_facts['Preprocessed_Facts'].tolist()

train_labels = df_train_facts['Decision'].tolist()
test_labels = df_test_facts['Decision'].tolist()

In [ ]:
#!pip install "numpy<2.0"  # or try "numpy==1.26.4"
#!pip install gensim

In [ ]:
from gensim.models import Word2Vec
from sklearn.preprocessing import scale
import nltk
nltk.download('punkt')  # Download tokenizer models
from nltk.tokenize import word_tokenize

In [ ]:
import numpy as np

In [ ]:
import nltk
nltk.download('punkt_tab')

In [ ]:
# Tokenization function
def tokenize_text(texts):
    """
    Tokenize a list of sentences into words.

    Parameters:
    - texts: List of raw sentences (strings)

    Returns:
    - List of tokenized sentences (lists of words)
    """
    return [word_tokenize(text.lower()) for text in texts]  # Lowercase and tokenize

# Word2Vec training function
def train_word2vec(sentences, embedding_dim=100, min_word_count=1, context=5):
    """
    Train a Word2Vec model on the given text data.

    Parameters:
    - sentences: List of tokenized sentences
    - embedding_dim: Dimensionality of word vectors
    - min_word_count: Minimum word count to include in the model
    - context: Context window size

    Returns:
    - Trained Word2Vec model
    """
    # Initialize and train the Word2Vec model
    model = Word2Vec(sentences, vector_size=embedding_dim, window=context, min_count=min_word_count, workers=4)
    return model

# Function to average word embeddings for a sentence
def get_average_word2vec(words, model, embedding_dim):
    """
    Average the Word2Vec vectors for a list of words.

    Parameters:
    - words: List of words in a sentence/document
    - model: Trained Word2Vec model
    - embedding_dim: Dimension of Word2Vec embeddings

    Returns:
    - Averaged Word2Vec feature vector for the sentence/document
    """
    # Initialize a zero vector
    feature_vec = np.zeros((embedding_dim,), dtype="float32")
    num_words = 0

    for word in words:
        if word in model.wv:
            num_words += 1
            feature_vec = np.add(feature_vec, model.wv[word])

    if num_words > 0:
        feature_vec = np.divide(feature_vec, num_words)

    return feature_vec

# Main function to tokenize and convert text data to Word2Vec features
def loadDataWord2Vec(X_train, X_test, embedding_dim=100, context=5):
    """
    Convert the training and test data into Word2Vec-based numerical features.

    Parameters:
    - X_train: List of raw training sentences
    - X_test: List of raw test sentences
    - embedding_dim: Dimensionality of word embeddings
    - context: Context window size for Word2Vec

    Returns:
    - X_train_w2v: Word2Vec-based numerical features for training data
    - X_test_w2v: Word2Vec-based numerical features for test data
    """
    # Step 1: Tokenize the raw sentences
    X_train_tokenized = tokenize_text(X_train)
    X_test_tokenized = tokenize_text(X_test)

    # Step 2: Train Word2Vec model on the tokenized sentences
    sentences = X_train_tokenized + X_test_tokenized  # Combine train and test data for Word2Vec training
    word2vec_model = train_word2vec(sentences, embedding_dim=embedding_dim, context=context)

    # Step 3: Convert each tokenized sentence into a numerical feature vector
    X_train_w2v = np.array([get_average_word2vec(sentence, word2vec_model, embedding_dim) for sentence in X_train_tokenized])
    X_test_w2v = np.array([get_average_word2vec(sentence, word2vec_model, embedding_dim) for sentence in X_test_tokenized])

    print(f"Word2Vec with {embedding_dim} features")
    return X_train_w2v, X_test_w2v

In [ ]:
# Convert to numerical features using TF-IDF
X_train_w2v, X_test_w2v = loadDataWord2Vec(train_text, test_text, embedding_dim=100)

In [ ]:
X_train_w2v[0]

In [ ]:
len(X_train_w2v)

In [ ]:
y_train=train_labels
y_test=test_labels

In [ ]:
from imblearn.metrics import specificity_score

from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    recall_score,
    precision_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
)


In [ ]:
# create a code for calculating model performance metrics and deliver it as one single data frame
def model_metrics(model, predictors, targets):
  pred=model.predict(predictors)
  pred_prob=model.predict_proba(predictors)[:,1]

  acc=accuracy_score(targets, pred)
  rec=recall_score(targets, pred)
  spec=specificity_score(targets, pred, average='binary')
  prec=precision_score(targets, pred)
  f1=f1_score(targets, pred)
  auc=roc_auc_score(targets,pred_prob)

  df_metrics=pd.DataFrame({}, index=['Metrics'])
  df_metrics['Accuracy']=acc
  df_metrics['Recall']=rec
  df_metrics['Specificity']=spec
  df_metrics['Precision']=prec
  df_metrics['F1']=f1
  df_metrics['AUC']=auc

  return df_metrics

In [ ]:
# Calculate and Display confusion matrix

def display_confusion_matrix(model, predictors, targets):
  pred=model.predict(predictors)
  cm=confusion_matrix(targets, pred)
  #cm_percentage=cm.astype('float')/cm.sum()*100
  disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1])
  disp.plot(cmap=plt.cm.Blues)
  plt.title("Confusion Matrix")
  plt.show()

## Training Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Step 1: Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Step 2: Train the model using X_train_w2v and y_train
rf_classifier.fit(X_train_w2v, y_train)

# Step 3: Make predictions on the test set
test_performance=model_metrics(rf_classifier, X_test_w2v, y_test)
test_performance

In [ ]:
display_confusion_matrix(rf_classifier, X_test_w2v, y_test)

In [ ]:
# Tune the model
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics
# defining model
Model =RandomForestClassifier(random_state=42, class_weight='balanced')

# Parameter grid to pass in RandomSearchCV
param_grid = {
    "n_estimators": [50,110,25],
    "min_samples_leaf": np.arange(1, 4),
    "max_features": [0.3, 0.4, 0.5, 'sqrt'],
    "max_samples": np.arange(0.4, 0.7, 0.1)
}

# Type of scoring used to compare parameter combinations
scorer = metrics.make_scorer(metrics.recall_score)

#Calling RandomizedSearchCV
randomized_cv_RF = RandomizedSearchCV(estimator=Model, param_distributions=param_grid, n_iter=10, n_jobs = -1, scoring=scorer, cv=5, random_state=1)

#Fitting parameters in RandomizedSearchCV
randomized_cv_RF.fit(X_train_w2v, y_train)

print("Best parameters are {} with CV score={}:" .format(randomized_cv_RF.best_params_,randomized_cv_RF.best_score_))

In [ ]:
best_params=randomized_cv_RF.best_params_
tuned_RF=RandomForestClassifier(random_state=42, class_weight='balanced', **best_params)
tuned_RF.fit(X_train_w2v, y_train)

In [ ]:
test_performance_tunedRF=model_metrics(tuned_RF, X_test_w2v, y_test)
test_performance_tunedRF.round(4)

## Training Gradient Boosting Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Step 1: Initialize the Gradient Boosting Classifier
gb_classifier = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# Step 2: Train the model using X_train_w2v and y_train
gb_classifier.fit(X_train_w2v, y_train)


# Step 3: Make predictions on the test set
test_performance=model_metrics(gb_classifier, X_test_w2v, y_test)
test_performance

In [ ]:
display_confusion_matrix(gb_classifier, X_test_w2v, y_test)

In [ ]:
# defining model

# To help with model building
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    AdaBoostClassifier,
    GradientBoostingClassifier,
    RandomForestClassifier,
    BaggingClassifier,
)
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression


Model =GradientBoostingClassifier(random_state=42)

# Parameter grid to pass in RandomSearchCV
param_grid = {
    "n_estimators": np.arange(50,110,25),
    "learning_rate": [0.01,0.1,0.05],
    "subsample":[0.7,0.9],
    "max_features":[0.5,0.7,1],
    "max_depth":[3,4,5],
}

# Type of scoring used to compare parameter combinations
scorer = metrics.make_scorer(metrics.recall_score)

#Calling RandomizedSearchCV
randomized_cv_GB = RandomizedSearchCV(estimator=Model, param_distributions=param_grid, n_iter=10, n_jobs = -1, scoring=scorer, cv=5, random_state=1)

#Fitting parameters in RandomizedSearchCV
randomized_cv_GB.fit(X_train_w2v, y_train)

print("Best parameters are {} with CV score={}:" .format(randomized_cv_GB.best_params_,randomized_cv_GB.best_score_))

In [ ]:
best_params=randomized_cv_GB.best_params_
tuned_GB=GradientBoostingClassifier(random_state=42,  **best_params)
tuned_GB.fit(X_train_w2v, y_train)

In [ ]:
test_performance_tunedGB=model_metrics(tuned_GB, X_test_w2v, y_test)
test_performance_tunedGB.round(4)

## Using XGBoost classifier

In [ ]:
!pip install xgboost

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Step 1: Initialize the XGBoost Classifier
xgb_classifier = xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# Step 2: Train the model using X_train_w2v and y_train
xgb_classifier.fit(X_train_w2v, y_train)

# Step 3: Make predictions on the test set
test_performance=model_metrics(xgb_classifier, X_test_w2v, y_test)
test_performance


In [ ]:
from sklearn import metrics

# defining model
Model = XGBClassifier(random_state=42)

# Parameter grid to pass in RandomSearchCV
param_grid={'n_estimators':np.arange(50,110,25),
            'scale_pos_weight':[1,2,5],
            'learning_rate':[0.01,0.1,0.05],
            'gamma':[1,3],
            'subsample':[0.7,0.9]
}

# Type of scoring used to compare parameter combinations
scorer = metrics.make_scorer(metrics.recall_score)

#Calling RandomizedSearchCV
randomized_cv_XGB = RandomizedSearchCV(estimator=Model, param_distributions=param_grid, n_iter=10, n_jobs = -1, scoring=scorer, cv=5, random_state=1)

#Fitting parameters in RandomizedSearchCV
randomized_cv_XGB.fit(X_train_w2v, y_train)

print("Best parameters are {} with CV score={}:" .format(randomized_cv_XGB.best_params_,randomized_cv_XGB.best_score_))

In [ ]:
best_params=randomized_cv_XGB.best_params_
tuned_XGB=XGBClassifier(random_state=42,  **best_params)
tuned_XGB.fit(X_train_w2v, y_train)

In [ ]:
test_performance_tunedXGB=model_metrics(tuned_XGB, X_test_w2v, y_test)
test_performance_tunedXGB.round(4)

## XGBoost- Using Oversampling/Undersampling Approach

In [ ]:
# To oversample and undersample data
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
print("Before Oversampling, counts of label '1': {}".format(sum(y_train)))
print("Before Oversampling, counts of label '0': {} \n".format(len(y_train)-sum(y_train)))

In [ ]:
#For oversampling
sm = SMOTE(
    sampling_strategy=1, k_neighbors=5, random_state=1
)  # Synthetic Minority Over Sampling Technique
X_train_over, y_train_over = sm.fit_resample(
  X_train_w2v, y_train)

In [ ]:
#for undersampling

rus = RandomUnderSampler(random_state=1)
X_train_un, y_train_un = rus.fit_resample(X_train_w2v, y_train)

In [ ]:
print("After Oversampling, counts of label '1': {}".format(sum(y_train_over)))
print("After Oversampling, counts of label '0': {} \n".format(len(y_train_over)-sum(y_train_over)))

In [ ]:
print("After undersampling, counts of label '1': {}".format(sum(y_train_un)))
print("After undersampling, counts of label '0': {} \n".format(len(y_train_un)-sum(y_train_un)))

In [ ]:
# Oversampling and XGB
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Step 1: Initialize the XGBoost Classifier
xgb_classifier = xgb.XGBClassifier(random_state=42,  **best_params)

# Step 2: Train the model using X_train_w2v and y_train
xgb_classifier.fit(X_train_over, y_train_over)

# Step 3: Make predictions on the test set
test_performance_xgbover=model_metrics(xgb_classifier, X_test_w2v, y_test)
test_performance_xgbover.round(4)

In [ ]:
# Undersampling and XGB
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Step 1: Initialize the XGBoost Classifier
xgb_classifier = xgb.XGBClassifier(random_state=42,  **best_params)

# Step 2: Train the model using X_train_w2v and y_train
xgb_classifier.fit(X_train_un, y_train_un)

# Step 3: Make predictions on the test set
test_performance_xgbunder=model_metrics(xgb_classifier, X_test_w2v, y_test)
test_performance_xgbunder.round(4)

In [ ]:
print(test_performance_tunedRF.round(4))
print(test_performance_tunedGB.round(4))
print(test_performance_tunedXGB.round(4))
print(test_performance_xgbover.round(4))
print(test_performance_xgbunder.round(4))